## Question 1:

In [4]:
import pandas as pd
import numpy as np

# 1. 讀取 CSV 檔案
df = pd.read_csv("googleplaystore.csv")

# 2. 建立一個函式來將 Size 欄位轉成數值 (單位：MB)
def convert_size_to_mb(size_str):
    """
    將 'Size' 欄位中的字串轉換為數值(以 MB 為單位)。
    - 若 size_str 含 'M'，移除 'M' 後轉成 float
    - 若 size_str 含 'k'，移除 'k' 後除以 1000 再轉成 float
    - 若值為 'Varies with device'、空白或無效，則回傳 np.nan
    """
    if not isinstance(size_str, str):
        return np.nan  # 不是字串就直接視為遺漏值

    size_str = size_str.strip()
    if size_str in ["Varies with device", "", "NaN"]:
        return np.nan
    if size_str[-1] == "M":
        # 例如 '19M' -> 19.0
        try:
            return float(size_str[:-1])
        except ValueError:
            return np.nan
    elif size_str[-1] == "k":
        # 例如 '700k' -> 0.7 MB
        try:
            return float(size_str[:-1]) / 1000.0
        except ValueError:
            return np.nan
    else:
        # 若沒有 'M'、'k'，可能是其他非預期字串
        # 也可能是一個單純數字，可以再視需求擴充
        try:
            return float(size_str)
        except ValueError:
            return np.nan

# 3. 將 Size 欄位轉換為數值單位 (MB)
df["Size_MB"] = df["Size"].apply(convert_size_to_mb)

# 4. 依照 Size_MB 從大到小排序，並選擇前 20 筆
df_sorted = df.sort_values(by="Size_MB", ascending=False)
top_20 = df_sorted.head(20)

# 5. 印出 App 名稱與對應的 Size
print("Top 20 Apps with the largest Size:")
for idx, row in top_20.iterrows():
    print(f"App: {row['App']}, Size: {row['Size']}, Size_MB: {row['Size_MB']}")

Top 20 Apps with the largest Size:
App: Mini Golf King - Multiplayer Game, Size: 100M, Size_MB: 100.0
App: Ultimate Tennis, Size: 100M, Size_MB: 100.0
App: Hungry Shark Evolution, Size: 100M, Size_MB: 100.0
App: SimCity BuildIt, Size: 100M, Size_MB: 100.0
App: Talking Babsy Baby: Baby Games, Size: 100M, Size_MB: 100.0
App: Draft Simulator for FUT 18, Size: 100M, Size_MB: 100.0
App: The Walking Dead: Our World, Size: 100M, Size_MB: 100.0
App: Stickman Legends: Shadow Wars, Size: 100M, Size_MB: 100.0
App: Post Bank, Size: 100M, Size_MB: 100.0
App: Car Crash III Beam DH Real Damage Simulator 2018, Size: 100M, Size_MB: 100.0
App: Hungry Shark Evolution, Size: 100M, Size_MB: 100.0
App: Vi Trainer, Size: 100M, Size_MB: 100.0
App: Miami crime simulator, Size: 100M, Size_MB: 100.0
App: Gangster Town: Vice District, Size: 100M, Size_MB: 100.0
App: Hungry Shark Evolution, Size: 100M, Size_MB: 100.0
App: Navi Radiography Pro, Size: 100M, Size_MB: 100.0
App: Rope Hero: Vice Town, Size: 99M, Size_M

## Question 2:

In [5]:
import pandas as pd
import numpy as np

# 1. 讀取 CSV 檔案
df = pd.read_csv("googleplaystore.csv")

# 2. 若檔案內有以字串形式存放的 "NaN" ，可先轉成 np.nan 便於後續統計
df.replace("NaN", np.nan, inplace=True)

# 3. 檢查各欄位是否存在缺失值
missing_counts = df.isnull().sum()

# 4. 列出有缺失值的欄位及缺失筆數
print("Attributes with missing values and their counts:")
for col in df.columns:
    if missing_counts[col] > 0:
        print(f"{col}: {missing_counts[col]}")

Attributes with missing values and their counts:
Rating: 1474
Type: 1
Content Rating: 1
Current Ver: 8
Android Ver: 3


## Question 3:

In [6]:
import pandas as pd
import numpy as np

# 1. 讀取 CSV 檔案
df = pd.read_csv("googleplaystore.csv")

# 2. 將 "Rating" 欄位轉成數值型態（若有無效字串則轉成 NaN）
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# 3. 先檢查 Rating 欄位的基礎資訊
print("===== Before Cleaning =====")
print("Rating column describe:\n", df["Rating"].describe())

# (1) 計算 Rating 的 mean、IQR、標準差
rating_mean_before = df["Rating"].mean()
rating_std_before = df["Rating"].std()
Q1_before = df["Rating"].quantile(0.25)
Q3_before = df["Rating"].quantile(0.75)
IQR_before = Q3_before - Q1_before

print(f"\n[Before] Mean: {rating_mean_before}")
print(f"[Before] IQR: {IQR_before} (Q1={Q1_before}, Q3={Q3_before})")
print(f"[Before] Std:  {rating_std_before}")

# (2) 識別異常值/錯誤值，並執行更正
# --------------------------------------------------
# 在實際情況下，你可以採用多種方式偵測或定義「異常值」 (outliers)，
# 這裡示範了兩種常見的方式（可以二擇一或結合）：
#
#  A. 依業務邏輯：有效評分應介於 1 ~ 5 之間（如果你確定沒有比 1 更低的評分系統）。
#  B. 使用 IQR (Interquartile Range) 規則：低於 (Q1 - 1.5 * IQR) 或高於 (Q3 + 1.5 * IQR) 的值可以視為潛在異常。
#
# 假設我們使用業務邏輯先進行基礎排除，再以 IQR 作最終檢查。

# A. 基於業務邏輯先排除範圍之外的值 (如：Rating < 1 或 Rating > 5)
condition_range = (df["Rating"] >= 1) & (df["Rating"] <= 5)

# B. 基於 IQR 計算潛在離群點
lower_bound = Q1_before - 1.5 * IQR_before
upper_bound = Q3_before + 1.5 * IQR_before
condition_iqr = (df["Rating"] >= lower_bound) & (df["Rating"] <= upper_bound)

# 將兩個條件結合；你可以選擇「同時符合兩者才保留」(AND)，或「符合任一者都保留」(OR)。
# 這裡示範採取“同時”符合，代表比較嚴格的過濾。
condition_final = condition_range & condition_iqr

# 找出「可疑異常」的資料
df_anomalies = df[~condition_final]
print("\n===== Potential anomalies (before correction) =====")
print(df_anomalies[["App", "Rating"]])

# 做法 1：直接將超出範圍或極端離群的 Rating 設為 NaN，再根據需求考慮是否插補或捨棄。
df.loc[~condition_final, "Rating"] = np.nan

# 做法 2：也可選擇「刪除整筆資料」(drop)。
# df = df[condition_final].copy()

# (這裡先示範做法 1：保留資料列，只把 Rating 無效值設為 NaN)

# (3) 更正後，重新計算 Mean, IQR, Std
df_after = df.dropna(subset=["Rating"])  # 先排除 Rating 為 NaN 才能正確計算

rating_mean_after = df_after["Rating"].mean()
rating_std_after = df_after["Rating"].std()
Q1_after = df_after["Rating"].quantile(0.25)
Q3_after = df_after["Rating"].quantile(0.75)
IQR_after = Q3_after - Q1_after

print("\n===== After Cleaning & Correction =====")
print("Rating column describe:\n", df_after["Rating"].describe())

print(f"[After] Mean: {rating_mean_after}")
print(f"[After] IQR:  {IQR_after} (Q1={Q1_after}, Q3={Q3_after})")
print(f"[After] Std:   {rating_std_after}")

===== Before Cleaning =====
Rating column describe:
 count    9367.000000
mean        4.193338
std         0.537431
min         1.000000
25%         4.000000
50%         4.300000
75%         4.500000
max        19.000000
Name: Rating, dtype: float64

[Before] Mean: 4.193338315362443
[Before] IQR: 0.5 (Q1=4.0, Q3=4.5)
[Before] Std:  0.5374313031477587

===== Potential anomalies (before correction) =====
                                   App  Rating
15     Learn To Draw Kawaii Characters     3.2
23              Mcqueen Coloring pages     NaN
87       RST - Sale of cars on the PCT     3.2
113          Wrinkles and rejuvenation     NaN
123             Manicure - nail design     NaN
...                                ...     ...
10824                        Cardio-FR     NaN
10825               Naruto & Boruto FR     NaN
10831         payermonstationnement.fr     NaN
10835                         FR Forms     NaN
10838           Parkinson Exercices FR     NaN

[1978 rows x 2 columns]

====

## Question 4:

In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# 1. 讀取 CSV 檔案
df = pd.read_csv("googleplaystore.csv")

# 2. 將 "Rating" 欄位轉為數值型態
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# 3. 若 "Price" 欄位是字串（例如 '$2.99'），需先去除符號，並轉為 float；若已經是數值可略過此步驟。
#    假設欄位名稱是 "Price"，且格式類似 "$2.99"、"$399.99" 或 "0"
def convert_price(p):
    if isinstance(p, str):
        # 去除 '$' 符號，若轉換失敗視為缺失
        p = p.replace("$", "").strip()
        try:
            return float(p)
        except ValueError:
            return np.nan
    else:
        return p

df["Price"] = df["Price"].apply(convert_price)

# 4. 建立分類(布林)欄位：
#   - Rating ≥ 4
#   - Price ≥ 100
df["Rating_4+"] = df["Rating"] >= 4
df["Price_100+"] = df["Price"] >= 100

# 5. 產生列聯表
contingency_table = pd.crosstab(df["Rating_4+"], df["Price_100+"])
print("Contingency table:\n", contingency_table)

# 6. 執行卡方檢定
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-square test statistic:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:\n", expected)

# 7. 判斷檢定結果 (以顯著水準 alpha = 0.05 為例)
alpha = 0.05
if p < alpha:
    print(f"\n檢定結果: p-value < {alpha}, 拒絕虛無假設(H0)")
    print("→ 推論：Rating≧4 與 Price≧100 之間具有統計上的關聯")
else:
    print(f"\n檢定結果: p-value >= {alpha}, 無法拒絕虛無假設(H0)")
    print("→ 推論：Rating≧4 與 Price≧100 之間沒有顯著統計關聯")

Contingency table:
 Price_100+  False  True 
Rating_4+               
False        3459     13
True         7362      7
Chi-square test statistic: 8.54722541926516
p-value: 0.003460492769636125
Degrees of freedom: 1
Expected frequencies:
 [[3.46559469e+03 6.40531316e+00]
 [7.35540531e+03 1.35946868e+01]]

檢定結果: p-value < 0.05, 拒絕虛無假設(H0)
→ 推論：Rating≧4 與 Price≧100 之間具有統計上的關聯
